In [2]:
import pickle
import numpy as np
from scipy.stats import entropy
from tqdm import tqdm
from datasets import load_dataset

In [3]:
dataset = load_dataset("potsawee/wiki_bio_gpt3_hallucination")
dataset = dataset['evaluation']

In [4]:
label_mapping = {
    'accurate': 0.0,
    'minor_inaccurate': 0.5,
    'major_inaccurate': 1.0,
}

In [5]:
human_label_detect_False   = {}
human_label_detect_False_h = {}
human_label_detect_True    = {}
for i_ in range(len(dataset)):
    dataset_i = dataset[i_]
    idx = dataset_i['wiki_bio_test_idx']
    raw_label = np.array([label_mapping[x] for x in dataset_i['annotation']])
    human_label_detect_False[idx] = (raw_label > 0.499).astype(np.int32).tolist()
    human_label_detect_True[idx]  = (raw_label < 0.499).astype(np.int32).tolist()
    average_score = np.mean(raw_label)
    if average_score < 0.99:
        human_label_detect_False_h[idx] = (raw_label > 0.99).astype(np.int32).tolist()

In [6]:
len(human_label_detect_False), len(human_label_detect_True), len(human_label_detect_False_h)

(238, 238, 206)

In [7]:
human_label_detect_False

{62464: [1, 1, 1, 1, 1, 1, 1, 1, 1],
 49661: [1, 1, 1, 1, 1, 1, 1],
 20483: [1, 1, 0, 1, 1, 1, 1, 1, 1],
 71174: [1, 1, 1, 1, 1, 1, 1, 1, 1],
 39945: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1],
 26126: [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1],
 61454: [1, 1, 1, 1, 1, 1, 1, 1, 1],
 37904: [1, 0, 0, 0, 0, 1, 1],
 61460: [1, 1, 1, 1, 0, 1, 1, 1],
 48151: [1, 1, 1, 1, 1, 1, 1, 1, 0],
 71192: [0, 0, 1, 1, 1, 0],
 17946: [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 21020: [1, 1, 0, 1, 1, 1, 1],
 20508: [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 13854: [1, 0, 1, 0, 1, 1, 1, 1, 1],
 1060: [1, 0, 1, 1, 1, 1, 0],
 72743: [1, 1, 1, 1, 1, 1],
 69672: [0, 0, 0, 0, 0],
 66602: [1, 1, 0, 1, 1, 1, 1],
 2091: [1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
 35884: [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 56366: [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 35887: [1, 0, 1, 0, 0],
 55347: [1, 0, 0, 1, 1, 1],
 4147: [1, 1, 1, 1, 1, 1, 1, 1, 1],
 30773: [1, 1, 1, 1, 1, 1, 1, 1],
 30775: [0, 0, 0, 0, 1, 0, 0, 0, 1],
 37431: [1, 1, 1, 1, 0, 1, 1, 1, 1],
 32313: [0, 